In [ ]:
# test_crawler.py
from playwright.sync_api import sync_playwright

def debug_crawler():
    with sync_playwright() as p:
        # 启动可视化浏览器
        browser = p.chromium.launch(headless=False)
        context = browser.new_context()
        page = context.new_page()
        
        try:
            # 录制操作
            page.goto("https://www.sciencedirect.com/journal/journal-of-computational-physics")
            page.wait_for_timeout(3000)  # 手动观察页面
            
            # 验证选择器
            test_selector = "article"
            elements = page.query_selector_all(test_selector)
            print(f"找到 {len(elements)} 篇文章")
            
            # 导出页面快照
            page.screenshot(path="debug_screenshot.png")
            with open("debug_page.html", "w") as f:
                f.write(page.content())
                
        finally:
            browser.close()

if __name__ == "__main__":
    debug_crawler()